In [8]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [56]:
from bs4 import BeautifulSoup
import requests
from datetime import datetime

headers = { 'User-Agent': 'Mozilla/5.0 (Windows NT 6.0; WOW64; rv:24.0) Gecko/20100101 Firefox/24.0' }
base_url = 'http://arkivet.operan.se/repertoar/list_productions.php?languageFlag=SE&titel=&programtitel=&upphov=&medverkande=&roll=&konstform=Opera&spelplats=&annanspelplats=&datum-ar=1770&datum-man=&datum-dag=&slutdatum-ar=2018&slutdatum-man=&slutdatum-dag=&sortering=1&currentPage='

In [58]:
listings = []

url = base_url + '1'
r  = requests.get(url, headers=headers).json()
repertoire_pages = r['resultInfo']['noOfPages']

#Get all operas
for repertoire_page in range(30, repertoire_pages + 1):
    print(repertoire_page)
    
    url = base_url + str(repertoire_page)
#     print(url)
    r  = requests.get(url, headers=headers).json()
    for production in r['productions']:

        if production['originalTitel']=='':
            work = production['name']
        else:
            work = production['originalTitel']
            
#         print(work)
        work_id = production['prodId']
    
        #Get details of the opera
        work_details_url = 'http://arkivet.operan.se/repertoar/list_authors.php?prodId=' + str(work_id)
        d = requests.get(work_details_url, headers=headers).json()
        composer = ''
        try:
            for author in d['authors']:
                if author['type']=='Musik':
                    composer = author['info']
        except:
            None
        
        #Get performances
        base_work_url = 'http://arkivet.operan.se/repertoar/list_shows.php?languageFlag=SE&titel=&programtitel=&upphov=&medverkande=&roll=&konstform=Opera&spelplats=&annanspelplats=&datum-ar=1770&datum-man=&datum-dag=&slutdatum-ar=2018&slutdatum-man=&slutdatum-dag=&sortering=1'
        base_work_url = base_work_url + '&prodId=' + str(work_id) + '&currentPage=' + str(repertoire_page) + '&currentShowsPage='
        work_url = base_work_url + '1'
#         print(work_url)
        p = requests.get(work_url, headers=headers).json()
        work_pages = p['resultInfo']['noOfPages']
        for work_page in range(1, work_pages+1):
#             print(work_pages)
            shows_url = base_work_url + str(work_page)
            s = requests.get(shows_url, headers=headers).json()
#             print(shows_url)
            shows = s['shows']
            for show in shows:
                date = show['date']
                date = datetime.strptime(date, '%Y-%m-%d').date()                
                listing = []
                listing.append(work)                        
                listing.append(composer)
                listing.append(date)
#                 print(listing)
                listings.append(listing)
    
#Saving results into dataframe and folder
import pandas as pd
headers = ['work', 'composer', 'date']
df = pd.DataFrame(listings, columns=headers)
df.to_csv('../../data/processed/listings/sweden_opera_house.csv')                                    

30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
